In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report
from scipy.stats.contingency import expected_freq
from scipy.special import xlogy
import plotly.express as px

In [ ]:
drive.mount('/content/gdrive')

In [ ]:
train = pd.read_parquet('gdrive/My Drive/Colab Notebooks/reddit_math_ds_train.parquet').reset_index(drop=True)
test = pd.read_parquet('gdrive/My Drive/Colab Notebooks/reddit_math_ds_test.parquet').reset_index(drop=True)
# train = pd.read_parquet('/Users/paul/data/reddit/reddit_math_ds_train.parquet').reset_index(drop=True)
# test = pd.read_parquet('/Users/paul/data/reddit/reddit_math_ds_test.parquet').reset_index(drop=True)

In [ ]:
train

In [ ]:
cv = CountVectorizer(stop_words='english', max_features=250)
count_model = cv.fit(train['text'])
train_counts = pd.DataFrame(cv.transform(train['text']).todense(), columns=cv.get_feature_names())
test_counts = pd.DataFrame(cv.transform(test['text']).todense(), columns=cv.get_feature_names())

## Decision tree classifiers

So far we have used information theory to measure the relevance of an input feature to a target variable.
With a little nudge we can use this to create a simple but powerful machine learning classifier called a _decision tree_.

### Warmup
The starting point is to view a single feature as a simple classifier.
In the dataset above, we can use the presence or absence of a given term to try to predict the subreddit label.
To train the classifier, we just have to decide which label we should apply if the term is present.

In [ ]:
pd.crosstab(train_counts['number'] != 0, train['subreddit'])

Let's treat the term `number` as a classifier by predicting that any reddit post containing `number` is in the `math` subreddit and all others are in `datascience`.

In [ ]:
classification_report(
    y_true=test['subreddit'] == 'math',
    y_pred=test_counts['number'] > 0,
    output_dict=True
)

Let's repeat this computation with all features:

In [ ]:
feature_df = pd.DataFrame(test_counts.columns, columns=['classifier'])

In [ ]:
feature_df['accuracy'] = feature_df['feature'].apply(
    lambda feature: classification_report(
        y_true=test['subreddit'] == 'math',
        y_pred=test_counts[feature] > 0,
        output_dict=True
    )['accuracy']
)

In [ ]:
feature_df.sort_values(by='accuracy', ascending=False)

#### Exercise
For each feature above, use the training data to compute the mutual information of the feature against the class label.
Plot mutual information against classifier accuracy, as above.
The code for computing mutual information based on a joint density table is below:

In [ ]:
def mutual_information(joint_density: pd.DataFrame) -> float:
    '''
    Input: DataFrame representing the joint density of a pair of random variables
    Output: Mutual information of the two random variables
    '''
    independent_density = pd.DataFrame(
        expected_freq(joint_density),
        columns=joint_density.columns,
        index=joint_density.index
    )
    return -(joint_density * np.log2(independent_density / joint_density)).sum().sum()

### Ensembling

The classifiers above have no hope of being very accurate because no one term appears in most of the posts.
To build a better classifier we'll need to ensemble them, so that the classifier considers more evidence.

A decision tree does this by assembling features into a chain of if/then statements; for instance:

- If "number" is present, output "math"
- If "number" is not present:
    - If "algebra" is present, output "math"
    - If "algebra" is not present, output "datascience"

Here we split the "number is not present" condition on the "engineering" feature; this allows our model to tell a story like "If the post doesn't contain the term 'number' and it does contain 'engineering' then it's probably in the datascience subreddit".

We could of course keep splitting, adding in more and more features as we go.
We can also split on the "number is present" branch of the tree, though that branch doesn't have much data so we'll get diminishing returns.

Here is an implementation of the decision tree above:

In [ ]:
pred = (test_counts['number'] > 0) | ((test_counts['number'] == 0) & (test_counts['algebra'] > 0))

In [ ]:
classification_report(
    y_true=test['subreddit'] == 'math',
    y_pred=pred,
    output_dict=True
)

#### Exercises
1. Repeat the computation above for all possible features in place of "algebra".  Which tree performs best?
2. Restrict the training data to include only posts not containing "number", and compute the mutual information between every feature and the class labels ("math" and "datascience").  How do these numbers compare with your accuracy numbers above?